#### From Christoph's email (see writeup), it seems reasonable to propose a measure of fitness $f = \left(\int \text{ LH}(a) da\right)^{\frac{1}{\text{life expectancy}}}.$ It has lots of desireable properties.

#### Let's see how it holds up in practice!

#### Using $P = [107.5, 0, 886]$ gives a distribution of nuts satisfying equal fitness between $\hat \beta = 0, 1$

#### So does using $P = [230.49,1000,886]$.

In [92]:
P = [107.5,0,886]
Q = [230.49,1000,886]
X = GeneralDiscreteDistribution(P)
Y = GeneralDiscreteDistribution(Q)
counts = [0]*len(P)
for _ in range(10000):
    counts[X.get_random_element()] += 1
print( [x/10000.n() for x in counts])

[0.108200000000000, 0.000000000000000, 0.891800000000000]


#### Run the life of a single squirrel, with nut finding distribution $\nu \sim Z$

In [40]:
def pregnancy_simulation(Z, offspring_threshold, n = 1000, verbose = False, life_history = False, all_details = False):
    nut_findings = [Z.get_random_element() for _ in range(n)]
    offspring_days = [0]*n
    nuts_in_storage = 0
    offspring_count = 0
    pregnant = False
    for k in range(n):
        nut_finding = nut_findings[k]
        nuts_in_storage += nut_finding - 1
        if nuts_in_storage < 0:
            break
        if pregnant:
            offspring_count += 1
            offspring_days[k] = 1
            pregnant = False
        if not pregnant and nuts_in_storage > offspring_threshold:
            pregnant = True
            nuts_in_storage += -1
    to_be_returned = None
    msg = 'Lived until {} - produced {} offspring'.format(k+1, offspring_count)
    bare_facts = (k+1, offspring_count)
    life_hist = offspring_days
    if verbose:
        return(msg)
    if life_history:
        return(life_hist)
    if all_details:
        return([bare_facts, life_hist])
    else:
        return(bare_facts)

In [47]:
pregnancy_simulation(X, 1)

(81, 64)

#### I need to know the life expectancy and the expected number of offspring produced.

In [48]:
def get_life_histories(Z, offspring_threshold, num_simulations = 1000):
    life_histories = []
    for i in range(num_simulations):
        life_histories.append(pregnancy_simulation(Z, offspring_threshold, life_history=True))
    return(life_histories)

def truncate_at_zeroes(some_list, num_zeroes = 5):
    support = [i for i, e in enumerate(some_list) if e != 0]
    truncation_point = max(support) + num_zeroes + 1
    truncated_list = deepcopy(some_list[:truncation_point])
    return(truncated_list)



def get_avg_num_offspring(life_histories):
    avg_num_offspring = [mean([lh[k] for lh in life_histories]).n() for k in range(len(life_histories[0]))]
    avg_num_offspring = truncate_at_zeroes(avg_num_offspring)
#    avg_num_offspring = [ (i, avg_num_offspring[i]) for i in range(len(avg_num_offspring))]
    return(avg_num_offspring)

def get_avg_life_history(Z, offspring_threshold, num_simulations = 1000):
    lh = get_life_histories(Z, offspring_threshold, num_simulations)
    aggregate_life_history = get_avg_num_offspring(lh)
    return(aggregate_life_history)

#### Sanity check: will summing the average life history give the average number of offspring produced? It should!!

In [64]:
sum(get_avg_life_history(X, 0, 100000))

7.32828999999999

In [65]:
num_offspring = []
for k in range(100000):
    n = pregnancy_simulation(X, 0)[1]
    num_offspring.append(n)

In [66]:
mean(num_offspring).n()

7.33685000000000

#### Good!! They are very close. Close enough that I am content to call this sanity check a success.

#### Now we are in a place to test whether squirrels with $\lambda_{\hat \beta = 0}\approx\lambda_{\hat \beta = 1}$ (Lyapunov exponents) have $f_{\hat \beta = 0}\approx f_{\hat \beta = 1}$ (our new measure of fitness).

In [91]:
get_life_expectancy(X, 1).n()

83.8830000000000

In [67]:
def get_life_expectancy(Z, offspring_threshold, num_simulations = 1000):
    death_day = []
    for _ in range(num_simulations):
        dd = pregnancy_simulation(Z, offspring_threshold)[0]
        death_day.append(dd)
    life_expectancy = mean(death_day)
    return(life_expectancy)

In [68]:
def approx_f(Z, offspring_threshold, num_simulations = 1000):
    RR = sum(get_avg_life_history(Z, offspring_threshold, num_simulations))
    LE = get_life_expectancy(Z, offspring_threshold, num_simulations)
    f = RR^(1/LE)
    return(f)

In [96]:
approx_f(X, 0)

1.23975777019737

In [73]:
approx_f(X, 1)

1.04978344554254

In [104]:
approx_f(Y, 0, num_simulations = 10000)

1.14402217131038

In [94]:
approx_f(Y, 1)

1.06325668672365